In [ ]:
# Users interacting with the v2/trades API need guidance on handling and manipulating JSON responses.

# A tutorial is required to show how to convert JSON into a tabular format for easier understanding and usability.

# This tutorial will be inserted into the developer portal for easy access.

In [ ]:
# JSON Handling: Provide clear instructions on manipulating JSON responses from the v2/trades API.

# Tabular Format: Include methods to convert JSON data into a tabular view for easier analysis.

# Tutorial Creation: Develop a tutorial with step-by-step instructions, code examples, and visual aids.

# Developer Portal Integration: Ensure the tutorial is published and accessible in the developer portal under the relevant section

In [ ]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import numpy as np

In [ ]:

#Step 1 : Make a simple API request.

URL='https://api.kpler.com/v2/cargo/trades?originDateStart=2024-07-01&originDateEnd=2024-07-31&originZones=Corpus Christi&products=LNG'

payload={}
headers = {'Authorization': 'Your-API-key'}

response=requests.get(URL, headers=headers, data=payload)
json_response=response.text
print(json_response) ## the output is a json string



In [ ]:
#Step 2 : Access the API response from the above request.

# Parse the json response and covert it to list
data =json.loads(json_response)

# Normalise the nested json in a flat table
df = json_normalize(data)

# Normalise the columns which contain lists of dictionaries
for column in df:

    if df[column].apply(lambda x: isinstance(x, list)).any():

       # Replace empty lists with NaN
        df.loc[df[column].apply(lambda x: isinstance(x, list) and len(x) == 0), column] = np.nan

    if df[column].apply(lambda x: isinstance(x, list) and len(x) > 0).any():

        # Explode the list of dictionaries in each row to separate rows
        df_exploded = df.explode(column)

        #normalise the columns
        column_df = json_normalize(df_exploded[column]).add_prefix(column +'.')

        #join the intial dataframe with the new one and drop any non normlised column
        df = df_exploded.drop(columns=[column]).join(column_df)


#Convert list to string using apply and join
for column in df:
     df[column] = df[column].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

#Deduplicate the final dataframe
df = df.drop_duplicates(keep='first')

#Print the final dataframe
print(df)

#Save the final dataframe to a csv
df.to_csv('Your-path')